The goal of this notebook is to generate all the debiased models used in the paper

This for experiment 1 is

- [X] HD gender
- [X] Sub same x3 - Sub (SCM)
- [X] LP same x3 - LP (SCM)
- [X] PP same x3 - GRA - SCM

In [ ]:
import os
import sys
import json
sys.path.insert(1,"../src/")
import config
import random
import numpy as np
import pandas as pd
from utils import load_embedding, load_pairs
from debias import Debias
from tqdm import tqdm
from eval import Evalution

## Loading Word Pairs

In [ ]:
dimension_pair_dict = {"gender":config.gender_pair_path, "race":config.race_pair_path, "age":config.race_pair_path}

## HD -- Only for gender

In [ ]:
debiasing_instance = Debias(config.sgns_wikitext_path, config.debiased_model_dir)

In [ ]:
hd = debiasing_instance.hard_debias()

## Debiasing with Subtraction (Sub) for each social attribute 

In [ ]:
for dim in dimension_pair_dict:
    print(dim)
    sub_embedding = debiasing_instance.subtraction(def_pairs=load_pairs(dimension_pair_dict[dim]))
    sub_embedding.save(os.path.join(config.debiased_model_dir, "sub_{}.wv".format(dim)))

## Debiasing with Linear Projection (LP) for each social attribute 

In [ ]:
for dim in dimension_pair_dict:
    print(dim)
    LP_embedding = debiasing_instance.projection(def_pairs=load_pairs(dimension_pair_dict[dim]))
    LP_embedding.save(os.path.join(config.debiased_model_dir, "LP_{}.wv".format(dim)))

## Debiasing with Partial Projection (PP) for each social attribute 

In [ ]:
for dim in dimension_pair_dict:
    print(dim)
    PP_embedding = debiasing_instance.partial_project(def_pairs=load_pairs(dimension_pair_dict[dim]))
    PP_embedding.save(os.path.join(config.debiased_model_dir, "PP_{}.wv".format(dim)))

## Repeating the Experiments with Random Word Pair Samples

The function "bootstrap" was used to measure ECT and EQT of Sub$_{same}$, LP$_{same}$, PP$_{same}$, and PP$_{SCM}$

In [ ]:
def bootstrap(deb_instance, dim_path_dict, repetitions=30, sub_sample_size=8):
    eval_instance = Evalution(profession_file_path= config.professions_path)
    rows  = {"alg":[], "dim":[], "repetition":[], "i":[], "sub_sample_size":[],"ect_correlation":[],
             "ect_pval":[], "eqt":[]}
    
    def get_measures(e, dim):
        if dim == "age":
            pairs = load_pairs(config.age_pair_path)
        elif dim == "race":
            pairs = load_pairs(config.race_pair_path)
        elif dim == "gender":
            pairs = load_pairs(config.gender_pair_path)
            
        #delete this
#         pairs = pairs[:2]
            
        ect = eval_instance.ect(embedding=e,pairs=pairs)
        ect_corr = ect.correlation
        ect_pval = ect.pvalue
        eqt_dict = eval_instance.eqt(embedding=e, pairs=pairs)
        eqt = np.average([eqt_dict[k] for k in eqt_dict])
        rows["ect_correlation"].append(ect_corr)
        rows["ect_pval"].append(ect_pval)
        rows["eqt"].append(eqt)

    def gather_info(i,alg, dim):
        rows["alg"].append(alg)
        rows["dim"].append(dim)
        rows["repetition"].append("repetition")
        rows["i"].append(i)
        rows["sub_sample_size"].append(sub_sample_size)
            
    for dim in dim_path_dict:
        pairs = load_pairs(dim_path_dict[dim])
        print(dim)
        for i in tqdm(range(repetitions)):
            sub_pairs = random.sample(pairs, sub_sample_size)
            e = deb_instance.subtraction(def_pairs = sub_pairs)
            gather_info(i, "sub", dim)
            get_measures(e, dim)
            e = debiasing_instance.projection(def_pairs=sub_pairs)
            gather_info(i,"LP", dim)
            get_measures(e, dim)
            e = debiasing_instance.partial_project(def_pairs=sub_pairs)
            gather_info(i,"PP",dim)
            get_measures(e,dim)
                    
    warmth_pairs = load_pairs(config.warmth_pair_path)
    competence_pairs = load_pairs(config.competence_pair_path)
    
    for i in tqdm(range(repetitions)):
        sub_warmth_pairs = random.sample(warmth_pairs, sub_sample_size)
        sub_competence_pairs = random.sample(competence_pairs, sub_sample_size)
        e = debiasing_instance.partial_project(def_pairs=sub_warmth_pairs)
        e_scm = debiasing_instance.partial_project(def_pairs=sub_competence_pairs, embedding=e)
        for dim in ["race", "age", "gender"]:
            gather_info(i, "PP_SCM", dim)
            get_measures(e_scm, dim)
        
            
    df = pd.DataFrame(rows)
    df.to_csv("../results/rep_{}_size_{}.csv".format(repetitions,sub_sample_size), index=False)
    return df

In [ ]:
rows = bootstrap(debiasing_instance, dim_path_dict = dimension_pair_dict, repetitions=1, sub_sample_size=2)

The function "bootstrap_scm" was used to repeatedly measure ECT and EQT, for Sub$_{SCM}$, LP$_{SCM}$, PP$_{SCM}$, and PP$_{G+R+A}$

In [ ]:
def bootstrap_scm(deb_instance, dim_path_dict, repetitions=30, sub_sample_size=8):
    eval_instance = Evalution(profession_file_path= config.professions_path)
    rows  = {"alg":[], "dim":[], "repetition":[], "i":[], "sub_sample_size":[],"ect_correlation":[],
             "ect_pval":[], "eqt":[]}
    
    def get_measures(e, dim):
        if dim == "age":
            pairs = load_pairs(config.age_pair_path)
        elif dim == "race":
            pairs = load_pairs(config.race_pair_path)
        elif dim == "gender":
            pairs = load_pairs(config.gender_pair_path)
            
        #delete this
#         pairs = pairs[:2]
            
        ect = eval_instance.ect(embedding=e,pairs=pairs)
        ect_corr = ect.correlation
        ect_pval = ect.pvalue
        eqt_dict = eval_instance.eqt(embedding=e, pairs=pairs)
        eqt = np.average([eqt_dict[k] for k in eqt_dict])
        rows["ect_correlation"].append(ect_corr)
        rows["ect_pval"].append(ect_pval)
        rows["eqt"].append(eqt)

    def gather_info(i,alg, dim):
        rows["alg"].append(alg)
        rows["dim"].append(dim)
        rows["repetition"].append("repetition")
        rows["i"].append(i)
        rows["sub_sample_size"].append(sub_sample_size)
            

    warmth_pairs = load_pairs(config.warmth_pair_path)
    competence_pairs = load_pairs(config.competence_pair_path)
    gender_pairs = load_pairs(config.gender_pair_path)
    race_pairs = load_pairs(config.race_pair_path)
    age_pairs = load_pairs(config.age_pair_path)
    
    for i in tqdm(range(repetitions)):
        sub_warmth_pairs = random.sample(warmth_pairs, sub_sample_size)
        sub_competence_pairs = random.sample(competence_pairs, sub_sample_size)
        sub_gender_pairs = random.sample(gender_pairs, sub_sample_size)
        sub_race_pairs = random.sample(race_pairs, sub_sample_size)
        sub_age_pairs = random.sample(age_pairs, sub_sample_size)
        
        # GRA
        eg = debiasing_instance.partial_project(def_pairs=sub_gender_pairs)
        e_gr = debiasing_instance.partial_project(def_pairs=sub_race_pairs, embedding=eg)
        e_gra = debiasing_instance.partial_project(def_pairs=sub_age_pairs, embedding=e_gr)
        for dim in ["race", "age", "gender"]:
            gather_info(i, "PP_GRA", dim)
            get_measures(e_gra, dim)
        
        # Sub_SCM
        e = debiasing_instance.subtraction(def_pairs=sub_warmth_pairs)
        e_scm = debiasing_instance.subtraction(def_pairs=sub_competence_pairs, embedding=e)
        for dim in ["race", "age", "gender"]:
            gather_info(i, "Sub_SCM", dim)
            get_measures(e_scm, dim)
        #LP_SCM
        e = debiasing_instance.projection(def_pairs=sub_warmth_pairs)
        e_scm = debiasing_instance.projection(def_pairs=sub_competence_pairs, embedding=e)
        for dim in ["race", "age", "gender"]:
            gather_info(i, "LP_SCM", dim)
            get_measures(e_scm, dim)
        #PP_SCM
        e = debiasing_instance.partial_project(def_pairs=sub_warmth_pairs)
        e_scm = debiasing_instance.partial_project(def_pairs=sub_competence_pairs, embedding=e)
        for dim in ["race", "age", "gender"]:
            gather_info(i, "PP_SCM", dim)
            get_measures(e_scm, dim)
            
        
            
    df = pd.DataFrame(rows)
    df.to_csv("../results/SCM_rep_{}_size_{}.csv".format(repetitions,sub_sample_size), index=False)
    return df

In [ ]:
scm_rows = bootstrap_scm(debiasing_instance, dim_path_dict = dimension_pair_dict, repetitions=1, sub_sample_size=2)


In [ ]:
scm_rows